In [199]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

output_file = '/Users/nonny/Desktop/25 July 2025/benchmarking-aba-dataset/Dataset 1/Version 1/Result-3-shot/Temp 0, top_p 0.05/output_formatted_reviews_gpt4o_3_v4_3.csv'
answer_key_file = 'ABA Dataset - Label Topic (For Tasks 1 & 2).xlsx'

df_output = pd.read_csv(output_file)
df_answer_key = pd.read_excel(answer_key_file)

In [200]:
df_answer_key.head(30)

,Unnamed: 0,Title,PositiveReview,NegativeReview,Topic,Selected Content,Pos/Neg,Head,Body 1,Body 2,...,Cont. Body 4,Cont. Body 5,Cont. Body 6,Cont. Body 7,Cont. Body 8,Cont. Body 9,Cont. Body 10,Concat,Unnamed: 29,Unnamed: 30
0,1,Wonderful place to stay.,"New, comfortable apartments, close to the airp...",Nothing at all.,Location,"close to the airport, to very clean beach.",Positive,good_location,close_to_airport,close_to_clean_beach,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"close_to_airport , close_to_clean_beach , no_e...",NaN,191.0
1,1,Wonderful place to stay.,"New, comfortable apartments, close to the airp...",Nothing at all.,Staff,Staff is extremely helpful and easy to communi...,Positive,good_staff,helpful,easy_to_communicate_with,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"helpful , easy_to_communicate_with , no_eviden...",NaN,NaN
2,1,Wonderful place to stay.,"New, comfortable apartments, close to the airp...",Nothing at all.,Food,"Tasty food on the first floor, comfortable res...",Positive,good_food,tasty_food,comfortable_restaurant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"tasty_food , comfortable_restaurant , no_evide...",NaN,NaN
3,2,It was superb,We had a really pleasant stay! The staff was v...,NaN,Staff,The staff was very nice and helpful!,Positive,good_staff,nice,helpful,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"nice , helpful , no_evident_not_nice , no_evid...",NaN,NaN
4,2,It was superb,We had a really pleasant stay! The staff was v...,NaN,Room,"The room was very clean, well decorated and mo...",Positive,good_room,clean_room,well_decorated,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"clean_room , well_decorated , modern , no_evid...",NaN,NaN
5,2,It was superb,We had a really pleasant stay! The staff was v...,NaN,Food,"Also, the breakfast was amazing, fresh and han...",Positive,good_food,amazing_breakfast,fresh_breakfast,...,no_evident_not_great_cooking,no_evident_not_kind_staff,no_evident_not_helpful,NaN,NaN,NaN,NaN,"amazing_breakfast , fresh_breakfast , handmade...",NaN,NaN
6,3,Very Good,the location is great and near the airport. bu...,NaN,Location,the location is great and near the airport. bu...,Positive,good_location,great,near_airport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"great , near_airport , bus_stop_is_close , no_...",NaN,NaN
7,4,Wonderful,Great stuff\n Great Quality/price\n Clean,NaN,Staff,Great stuff,Positive,good_staff,great,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"great , no_evident_not_great",NaN,NaN
8,4,Wonderful,Wonderful,NaN,Price,Great Quality/price,Positive,good_price,great_quality,great_price,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"great_quality , great_price , no_evident_not_g...",NaN,NaN
9,4,Wonderful,Wonderful,NaN,Room,Clean,Positive,good_room,clean,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"clean , no_evident_not_clean",NaN,NaN


In [201]:
# Clean answer key
df_answer_cleaned = df_answer_key[['Unnamed: 0', 'Topic', 'Selected Content', 'Pos/Neg']].rename(columns={
    'Unnamed: 0': 'ID',
    'Topic': 'Topics',
    'Selected Content': 'Text',
    'Pos/Neg': 'NegPos'
})

# df_answer_cleaned.head(30)

In [202]:
# Clean Output
def clean_topic(topic):
    return re.sub(r'\d+$', '', str(topic))

df_output['Cleaned_Topics'] = df_output['Topics'].apply(clean_topic)

df_output_cleaned = df_output[['ID', 'Cleaned_Topics', 'Text', 'NegPos']].copy()
df_output_cleaned.rename(columns={'Cleaned_Topics': 'Topics'}, inplace=True)
df_output_cleaned = df_output_cleaned[~(df_output_cleaned['Text'].isna() & df_output_cleaned['NegPos'].isna())]

# df_output_cleaned.head(30)

df_output_merged = df_output_cleaned.groupby(['ID', 'Topics', 'NegPos']).agg({
    'Text': lambda x: ', '.join(x.dropna().astype(str).str.strip())
}).reset_index()


# df_output_merged.head(30)
df_output_merged[df_output_merged['ID'] == 22]


,ID,Topics,NegPos,Text
60,22,Facility,Positive,The restaurant area is nice.
61,22,Food,Negative,"Ordered pizza one night and was not great, too..."
62,22,Food,Positive,I'm sure other food is better.
63,22,Room,Negative,might have needed another set of drawers if th...
64,22,Room,Positive,"Room was perfect, looked new, clean shiny tile..."
65,22,Staff,Positive,The staff very accommodating.


In [203]:
df_compare = pd.merge(
    df_answer_cleaned.rename(columns={'NegPos': 'NegPos_answer', 'Text': 'Text_answer'}),
    df_output_merged.rename(columns={'NegPos': 'NegPos_output', 'Text': 'Text_output'}),
    left_on=['ID', 'Topics', 'NegPos_answer'],
    right_on=['ID', 'Topics', 'NegPos_output'],
    how='outer'
)


df_compare.head(30)
# df_compare[df_compare['ID'] == 22]

,ID,Topics,Text_answer,NegPos_answer,NegPos_output,Text_output
0,1,Food,"Tasty food on the first floor, comfortable res...",Positive,Positive,"Tasty food on the first floor, comfortable res..."
1,1,Location,"close to the airport, to very clean beach.",Positive,Positive,"close to the airport, to very clean beach."
2,1,Room,NaN,NaN,Positive,"New, comfortable apartments."
3,1,Staff,Staff is extremely helpful and easy to communi...,Positive,Positive,Staff is extremely helpful and easy to communi...
4,2,Facility,NaN,NaN,Positive,"I like hotels with a family atmosphere, cozy a..."
5,2,Food,"Also, the breakfast was amazing, fresh and han...",Positive,Positive,"the breakfast was amazing, fresh and handmade!..."
6,2,Room,NaN,NaN,Negative,although not big.
7,2,Room,"The room was very clean, well decorated and mo...",Positive,Positive,"The room was very clean, well decorated and mo..."
8,2,Staff,The staff was very nice and helpful!,Positive,Positive,The staff was very nice and helpful. The cook ...
9,3,Location,the location is great and near the airport. bu...,Positive,Positive,the location is great and near the airport. bu...


In [204]:
def sentiment_match(row):
    if pd.isna(row['NegPos_answer']) or pd.isna(row['NegPos_output']):
        return False  # Missing either → False
    return str(row['NegPos_answer']).strip().lower() == str(row['NegPos_output']).strip().lower()

df_compare['Sentiment_Match'] = df_compare.apply(sentiment_match, axis=1)

df_compare.head(30)
# df_compare[df_compare['ID'] == 22]

,ID,Topics,Text_answer,NegPos_answer,NegPos_output,Text_output,Sentiment_Match
0,1,Food,"Tasty food on the first floor, comfortable res...",Positive,Positive,"Tasty food on the first floor, comfortable res...",True
1,1,Location,"close to the airport, to very clean beach.",Positive,Positive,"close to the airport, to very clean beach.",True
2,1,Room,NaN,NaN,Positive,"New, comfortable apartments.",False
3,1,Staff,Staff is extremely helpful and easy to communi...,Positive,Positive,Staff is extremely helpful and easy to communi...,True
4,2,Facility,NaN,NaN,Positive,"I like hotels with a family atmosphere, cozy a...",False
5,2,Food,"Also, the breakfast was amazing, fresh and han...",Positive,Positive,"the breakfast was amazing, fresh and handmade!...",True
6,2,Room,NaN,NaN,Negative,although not big.,False
7,2,Room,"The room was very clean, well decorated and mo...",Positive,Positive,"The room was very clean, well decorated and mo...",True
8,2,Staff,The staff was very nice and helpful!,Positive,Positive,The staff was very nice and helpful. The cook ...,True
9,3,Location,the location is great and near the airport. bu...,Positive,Positive,the location is great and near the airport. bu...,True


In [205]:
df_compare.to_csv('sentiment_result_3_shot_3.csv', index=False)

In [206]:
# Calculate accuracy
num_true = df_compare['Sentiment_Match'].sum()
num_total = len(df_compare)

accuracy = num_true / num_total

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Correct (True): {num_true}")
print(f"Total Cases: {num_total}")


Accuracy: 56.23%
Correct (True): 352
Total Cases: 626


In [207]:
# True Positive: sentiment predicted and matched
TP = df_compare[(df_compare['NegPos_answer'].notna()) & (df_compare['NegPos_output'].notna()) & (df_compare['Sentiment_Match'])].shape[0]

# False Positive: prediction exists but no match or wrong
FP = df_compare[(df_compare['NegPos_output'].notna()) & (~df_compare['Sentiment_Match'])].shape[0]

# False Negative: ground truth exists but no prediction or wrong
FN = df_compare[(df_compare['NegPos_answer'].notna()) & (~df_compare['Sentiment_Match'])].shape[0]

# Metrics
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Precision: 0.5896
Recall: 0.9239
F1 Score: 0.7198
